In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [5]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


En este bloque se importan las librerías necesarias para el preprocesamiento realizado.
Pandas y numpy para manipular datos, SimpleImputer y StandardScaler para imputar y escalar valores numéricos.

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import os


Aquí se carga el archivo train.csv y se realiza una exploración básica.
Se muestran las dimensiones del dataset, las primeras filas, la información de las columnas (tipos de datos) y el conteo de valores faltantes.
Esto permite entender la estructura general de los datos antes de realizar el preprocesamiento.

In [7]:
df = pd.read_csv("train.csv")
print("Dimensiones del dataset:", df.shape)
df.head()


Dimensiones del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [8]:
print("Información general:")
df.info()

print("\nValores faltantes por columna:")
print(df.isnull().sum())

Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 n

Este bloque separa las variables del dataset según su tipo:
las columnas numéricas (valores continuos o discretos) y las categóricas (texto o categorías).
Esta clasificación es importante porque cada tipo de dato se procesará con una técnica diferente (por ejemplo, imputación o codificación).

In [9]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns

print("Columnas numéricas:", list(num_cols))
print("Columnas categóricas:", list(cat_cols))


Columnas numéricas: ['ID', 'PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
Columnas categóricas: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']


Los modelos no pueden trabajar con datos vacíos.
Por eso, en este bloque se completan los valores faltantes:

Para las columnas numéricas, se reemplazan los valores nulos por la media.

Para las columnas categóricas, se reemplazan por el valor más frecuente (moda).
Esto asegura que el dataset quede completo y listo para las transformaciones posteriores.

In [10]:
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

df[num_cols] = num_imputer.fit_transform(df[num_cols])
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])


Las columnas categóricas contienen textos que los algoritmos no pueden procesar directamente.
Con el One-Hot Encoding convertimos cada categoría en una columna nueva con valores 1 o 0, indicando la presencia o ausencia de esa categoría.


In [13]:
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

print("Dimensiones después del One-Hot Encoding:", df.shape)


Dimensiones después del One-Hot Encoding: (692500, 1032)


En este paso se normalizan las columnas numéricas para que todas estén en una escala comparable.
Algunos algoritmos de machine learning son sensibles a las diferencias de magnitud entre variables, por lo que la normalización (media 0 y desviación estándar 1) ayuda a mejorar el rendimiento y estabilidad del modelo.

In [14]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

print("Ejemplo de datos normalizados:")
display(df[num_cols].head())


Ejemplo de datos normalizados:


,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,1.434424,1.294094,0.437002,-0.556223,0.813978,0.060296
1,0.527513,1.294094,0.346934,-0.481341,0.508180,0.016142
2,-0.652132,0.439801,0.232301,-0.492038,0.729034,0.016142
3,-0.084924,-0.319570,1.771650,-0.941332,-0.171371,-1.072993
4,1.731274,1.294094,0.387874,-0.299484,0.389259,0.457683


In [ ]:
file_size = os.path.getsize("train_preprocesado.csv") / (1024 * 1024)
print(f"Tamaño del archivo procesado: {file_size:.2f} MB")

preview = pd.read_csv("train_preprocesado.csv", nrows=10)
print("\nVista previa de las primeras 10 filas del archivo guardado:")
display(preview)
